In [12]:
import pypsa
import pandas as pd

In [3]:
n = pypsa.Network("D:/project_h2/Results/4/2020.nc")

INFO:pypsa.io:Imported network 2020.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [4]:
# Sum energy output of all generators over time
gen_energy = n.generators_t.p.sum()

# Group by generator carrier and sum energy
gen_energy_by_carrier = gen_energy.groupby(n.generators.carrier).sum()

# Convert to percentage
gen_shares = (gen_energy_by_carrier / gen_energy_by_carrier.sum()) * 100

# Print rounded results
print("Generator energy supply shares:")
print(gen_shares.round(2).astype(str) + ' %')


Generator energy supply shares:
carrier
H2              0.0 %
biomass        5.83 %
coal           4.46 %
gas           49.74 %
lignite        9.33 %
load            0.0 %
offwind-ac     1.07 %
offwind-dc     1.42 %
oil            0.02 %
onwind        17.38 %
ror            1.88 %
solar          8.86 %
dtype: object


Total heat distribution

In [10]:
heating_links = []

for link_name, row in n.links.iterrows():
    if isinstance(row.get("bus1", ""), str) and "heat" in row["bus1"]:
        heating_links.append(link_name)
    if isinstance(row.get("bus2", ""), str) and "heat" in row["bus2"]:
        heating_links.append(link_name)

heating_links_df = n.links.loc[heating_links]
# Sum heat delivered by carrier (not per link)
p_by_carrier = heating_links_df.groupby("carrier").apply(
    lambda x: sum([
        -n.links_t.p1[link].sum() if 'heat' in n.links.loc[link, 'bus1']
        else -n.links_t.p2[link].sum()
        for link in x.index
    ])
).astype(float)

# Normalize to percentage
supply_share = 100 * p_by_carrier / p_by_carrier.sum()
print("\nHeating supply shares by technology:")
print(supply_share.round(2).astype(str) + " %")



Heating supply shares by technology:
carrier
AC         58.32 %
CCGT       10.11 %
biomass      0.0 %
gas        31.57 %
oil          0.0 %
dtype: object


In [13]:
heating_links = []

for link_name, row in n.links.iterrows():
    if isinstance(row.get("bus1", ""), str) and "heat" in row["bus1"]:
        heating_links.append(link_name)
    if isinstance(row.get("bus2", ""), str) and "heat" in row["bus2"]:
        heating_links.append(link_name)

p_by_link = {}

for link in heating_links:
    if "bus1" in n.links.columns and "heat" in n.links.at[link, "bus1"]:
        p_by_link[link] = -n.links_t.p1[link].sum()
    elif "bus2" in n.links.columns and "heat" in n.links.at[link, "bus2"]:
        p_by_link[link] = -n.links_t.p2[link].sum()

p_by_link = pd.Series(p_by_link)
p_by_link = p_by_link[(p_by_link / p_by_link.sum()) > 0.01]


print("Heating supply shares by link:")
print((p_by_link / p_by_link.sum() * 100).round(2).astype(str) + ' %')


Heating supply shares by link:
DE0 2 CCGT           3.32 %
DE0 A CCGT           6.79 %
DE0 9 heat_pump     14.29 %
DE0 A heat_pump     14.94 %
DE0 2 heat_pump     15.08 %
DE0 E heat_pump     13.99 %
DE0 9 gas_boiler     3.99 %
DE0 A gas_boiler     13.4 %
DE0 2 gas_boiler    10.78 %
DE0 E gas_boiler      3.4 %
dtype: object


total heat demand supplied per bus

In [18]:
# Initialize an empty series for accumulating heat supply
heat_supply_total = pd.Series(dtype=float)

# 1. Links where heat is delivered at bus1 (majority of heating techs)
bus1_heat_links = n.links[n.links.bus1.str.contains("heat", case=False, na=False)]
heat_by_bus1 = -n.links_t.p1[bus1_heat_links.index].sum()
bus1_map = bus1_heat_links["bus1"]
heat_by_bus1_grouped = heat_by_bus1.groupby(bus1_map).sum()

# 2. Links where heat is delivered at bus2 (e.g., CHP like CCGT)
bus2_heat_links = n.links[n.links.bus2.str.contains("heat", case=False, na=False)]
heat_by_bus2 = n.links_t.p2[bus2_heat_links.index].sum()
bus2_map = bus2_heat_links["bus2"]
heat_by_bus2_grouped = heat_by_bus2.groupby(bus2_map).sum()

# Combine both
heat_supply_total = heat_by_bus1_grouped.add(heat_by_bus2_grouped, fill_value=0)

# Display sorted
heat_supply_total = heat_supply_total.sort_values(ascending=False)
heat_supply_total.name = "Total Heat Supplied (MWh)"

In [19]:
heat_supply_total

bus1
DE0 2 heat    1.710921e+08
DE0 A heat    1.636153e+08
DE0 9 heat    1.387469e+08
DE0 E heat    1.320231e+08
Name: Total Heat Supplied (MWh), dtype: float64